In [1]:
import sys
sys.path.append("../src")
import pandas as pd
import Preprocessing

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
# read the dataset
df_full_preprocessed = pd.read_pickle("./df_full_preprocessed.pkl")
# import preprocessor
preprocessing = Preprocessing.Preprocessing()

In [3]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(df_full_preprocessed)**(1/4)), use_idf=True)
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), ["gender","topic","sign"])\
                                                           , remainder=StandardScaler())


In [4]:
# create dataset
X_text = text_transformer.fit_transform(df_full_preprocessed["text_preprocessed"])
X_numerical = numerical_transformer.fit_transform(df_full_preprocessed.drop("text_preprocessed", axis=1))

In [5]:
knn_text = NearestNeighbors(n_neighbors=10)
knn_text.fit(X_text)

NearestNeighbors(n_neighbors=10)

In [6]:
knn_numerical = NearestNeighbors(n_neighbors=10)
knn_numerical.fit(X_numerical)

NearestNeighbors(n_neighbors=10)

In [7]:
knn_pipeline = pd.Series([text_transformer, numerical_transformer, knn_text, knn_numerical])

In [8]:
knn_pipeline.to_pickle("./Pipelines/KNearestNeighborsPipeline.pkl")